In [228]:
import os
import json
import numpy as np
import pandas as pd
from openai import OpenAI
import random
import time
import openai

In [294]:
api_key = Insert key
client = OpenAI(api_key=api_key)

In [510]:
username_label = '''

Analyze username information to infer:

"name_origin": the user's identity or likely ethnic background (0: Chinese-American, 1: Korean-American, 2: Japanese-American, 3: Other Asian-American, 4: Hispanic/Latino-American, 5: Other, 6: Unsure).
Output as JSON.

'''

prompt_word_count = len(username_label.split())
prompt_word_count

In [512]:

df = pd.read_csv('../../../qss45_data/shohei-ig-cleaned.csv')



In [489]:
df.lang.value_counts()

lang
ja       9171
en       7661
es       5282
other     757
zh        353
ko         20
Name: count, dtype: int64

In [490]:
df = df.dropna(subset=['username'])
df['id'] = df['id'].astype(str)

In [491]:
df = df[df['lang'] == 'en']
df = df[['id', 'username']]

In [492]:
df.shape

(7575, 2)

In [494]:
# test that classifying works properly
# keywords = ['kim', 'park', 'lee', 'cho', 'kang', 'yang']

# # Use str.contains to filter the 'username' column for any of the keywords
# filtered_df = df[df['username'].str.contains('|'.join(keywords), case=False, na=False)]
# filtered_df = filtered_df[~filtered_df['username'].str.contains('mikeybleedsblue', case=False, na=False)]

# # Display the value counts of the filtered usernames
# filtered_df['username'].value_counts()

In [495]:
# df_2 = df.tail(228)
# df_2

,id,username
22492,1067086895037795,David John Pollard ohtanicanada
22496,1347869739920335,❤︎Sho0705Time❤︎ __sho0705time__
22498,909575060622560,MLBCN mlbcoveragenews
22499,573108991852780,Nate dodgernate
22502,560580903130000,Joshua Shiffman js9innings
...,...,...
23222,1333850967577362,TheRealDodgerHogs therealdodgerhogs
23227,1264224878366252,❤︎Sho0705Time❤︎ __sho0705time__
23228,563515409501409,Cinemonie thecinemonie
23229,911617997574545,David John Pollard ohtanicanada


In [496]:
df_2.loc[:, 'nwords'] = df_2['username'].apply(lambda x: len(x.split()))
df_2.loc[:, 'nwords'] += prompt_word_count  # Add the prompt word count to each row
df_2.loc[:, 'ntokens'] = df_2['nwords'] / 0.7  # Conservative token estimation

# Token limits
max_tokens_per_subset = 4096
total_tokens = df_2['ntokens'].sum()

# Split the DataFrame if total tokens exceed the limit
if total_tokens > max_tokens_per_subset:
    split_df = np.array_split(df_2, 20)
else:
    split_df = [df_2]

/var/folders/62/q5cwr8p541q40q2zbtg90hxh0000gn/T/ipykernel_58008/1596290570.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'nwords'] = df_2['username'].apply(lambda x: len(x.split()))
/var/folders/62/q5cwr8p541q40q2zbtg90hxh0000gn/T/ipykernel_58008/1596290570.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2.loc[:, 'ntokens'] = df_2['nwords'] / 0.7  # Conservative token estimation
/Users/gabriellepark/venv/test_env/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:57: FutureWar

In [497]:
split_df[19]

,id,username,nwords,ntokens
23204,580557580978003,CAA Baseball caabaseball,34,48.571429
23206,898696155179176,❤︎Sho0705Time❤︎ __sho0705time__,33,47.142857
23208,1186095329159429,David John Pollard ohtanicanada,35,50.000000
23215,787712483423317,David John Pollard ohtanicanada,35,50.000000
23216,2003002253482954,Cinemonie thecinemonie,33,47.142857
23221,466323556461769,Joshua Shiffman js9innings,34,48.571429
23222,1333850967577362,TheRealDodgerHogs therealdodgerhogs,33,47.142857
23227,1264224878366252,❤︎Sho0705Time❤︎ __sho0705time__,33,47.142857
23228,563515409501409,Cinemonie thecinemonie,33,47.142857
23229,911617997574545,David John Pollard ohtanicanada,35,50.000000


In [279]:
# model_name = "gpt-4-turbo"

In [498]:
def get_tasks(df):
    tasks_this_batch = []
    for index, row in df.iterrows():
        description = row['username']
        task = {
            "custom_id": str(row['id']),
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                "model": "gpt-3.5-turbo",
                "max_tokens": 200,
                "temperature": 0,
                "response_format": {"type": "json_object"},
                "messages": [
                    {"role": "system", "content": username_label}, # gets row
                    {"role": "user", "content": description},
                ],
            },
        }
        tasks_this_batch.append(task)
    return tasks_this_batch

output_folder = "batch_input"
for i, batch in enumerate(split_df):
    tasks = get_tasks(batch)
    file_name = f"batch_tasks_b{i}_4o-mini.json"
    file_path = os.path.join(output_folder, file_name)
    with open(file_path, 'w') as file:
        for task in tasks:
            json.dump(task, file)
            file.write('\n')


In [500]:
batch_ids = list(range(0, 20))
batch_ids


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [501]:
batch_jobs = []

for batch_num in batch_ids:

    file_name = f"{output_folder}/batch_tasks_b{batch_num}_4o-mini.json"
    batch_input_file = client.files.create(
        file=open(file_name, "rb"),
        purpose="batch"
    )
    batch_input_file_id = batch_input_file.id
    # Create batch
    batch_job = client.batches.create(
        input_file_id=batch_input_file_id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={"description": f"batch_b{batch_num}_4o-mini"}
    )
    checking_batch_one = client.batches.retrieve(batch_job.id)
    print(f"Batch {batch_num} status: {checking_batch_one.status}")
    batch_jobs.append(checking_batch_one.id)

    

Batch 0 status: validating
Batch 1 status: validating
Batch 2 status: validating
Batch 3 status: validating
Batch 4 status: validating
Batch 5 status: validating
Batch 6 status: validating
Batch 7 status: validating
Batch 8 status: validating
Batch 9 status: validating
Batch 10 status: validating
Batch 11 status: validating
Batch 12 status: validating
Batch 13 status: validating
Batch 14 status: validating
Batch 15 status: validating
Batch 16 status: validating
Batch 17 status: validating
Batch 18 status: validating
Batch 19 status: validating


In [503]:
print("All batch IDs:", batch_jobs)


All batch IDs: ['batch_6745f7920e1c8190af757e561ddf19e2', 'batch_6745f793de288190a1d798019b9481eb', 'batch_6745f79524c881908b64e69054e35f27', 'batch_6745f79699f081908177a47666168d20', 'batch_6745f797cffc8190ae1c37aee5643eea', 'batch_6745f7990d38819080052d10f82c44ad', 'batch_6745f79a2b548190af56b72847efe1a9', 'batch_6745f79b63b48190b8555622fe5aa72f', 'batch_6745f79c9fd48190ac840cffd32c2c18', 'batch_6745f79d90cc8190b566298ba85b06d1', 'batch_6745f79e775081909f61bacf0d0fd9b0', 'batch_6745f79ff744819086308a57e2bf6eec', 'batch_6745f7a117008190b989d470fce46978', 'batch_6745f7a1ef288190afaa2f85583fb79a', 'batch_6745f7a2ebac8190bb12f324bc3f3bc2', 'batch_6745f7a42f6881908a9020d3a6194d9a', 'batch_6745f7a50d9c81909d6ae49bc323ff85', 'batch_6745f7a63e1c8190b1a4b2d0c8bf2b75', 'batch_6745f7a7237081909446e5dca9ef6123', 'batch_6745f7a81f2081908ee767807fccd296']


In [502]:
for id in batch_jobs:
    

    checking_batch_one = client.batches.retrieve(id)
    print(checking_batch_one.status)


completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed
completed


In [389]:
# client.batches.retrieve(batch_job.id)

Batch(id='batch_6745d2710bf881908e8e0549127c1cbb', completion_window='24h', created_at=1732629105, endpoint='/v1/chat/completions', input_file_id='file-K3cZEhS11PcN9MQmqZqVdC', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1732629249, error_file_id=None, errors=None, expired_at=None, expires_at=1732715505, failed_at=None, finalizing_at=1732629230, in_progress_at=1732629106, metadata={'description': 'batch_b0_4o-mini'}, output_file_id='file-WtKyubGRYzsxy6NQaGCDzx', request_counts=BatchRequestCounts(completed=150, failed=0, total=150))

In [402]:
def extract_name_origin(content):
        try:
            return json.loads(content).get('name_origin', None)
        except (json.JSONDecodeError, TypeError):
            return None  

In [504]:
all_results_df = pd.DataFrame()

for batch_num in range(0, 19):
    idnum = batch_jobs[batch_num]
    batch_job = client.batches.retrieve(idnum)
    result_file_id = batch_job.output_file_id

    file_response = client.files.content(result_file_id)
        
    # Decode and split the file content
    result = file_response.content.decode("utf-8")
    lines = result.splitlines()
        
    # Parse each line of the file into a JSON object
    result_jsons = [json.loads(line) for line in lines]
        
        # Optionally, process the `result_jsons` here, e.g., save or analyze the results
    print(f"Batch {batch_num} results: proccessed")
    output = []
    output_all = []

    for obj in result_jsons:
        output_all.append(obj)
        # Extracting id and content from the response
        article = {"id": obj['custom_id'], "content": obj['response']['body']['choices'][0]['message']['content']}
        output.append(article)

    # Save the simplified output (only id and content)
    output_filename = f"{output_folder}/batch_b{batch_num}_output.json"
    with open(output_filename, "w") as file:
        json.dump(output, file, indent=4)
    result_file_name = f"{output_folder}/batch_b{batch_num}_output_full.json"
    with open(result_file_name, "w") as file:
        json.dump(output_all, file, indent=4)

    # Convert the output into a DataFrame
    df_results = pd.DataFrame(output)
    df_results['name_origin'] = df_results['content'].apply(extract_name_origin)
    print(df_results.head(1))
    print(f"Batch {batch_num} processed:")

    # Append the results to the all_results_df DataFrame
    all_results_df = pd.concat([all_results_df, df_results[['id', 'name_origin']]], ignore_index=True)


Batch 0 results: proccessed
                 id                   content  name_origin
0  1067086895037795  {\n  "name_origin": 5\n}            5
Batch 0 processed:
Batch 1 results: proccessed
                id                   content  name_origin
0  904655944541533  {\n  "name_origin": 5\n}            5
Batch 1 processed:
Batch 2 results: proccessed
                id                   content  name_origin
0  502170179481382  {\n  "name_origin": 5\n}            5
Batch 2 processed:
Batch 3 results: proccessed
                 id                   content  name_origin
0  2003328033445328  {\n  "name_origin": 2\n}            2
Batch 3 processed:
Batch 4 results: proccessed
                 id                   content  name_origin
0  1312560203486775  {\n  "name_origin": 5\n}            5
Batch 4 processed:
Batch 5 results: proccessed
                 id                     content  name_origin
0  1225411172018889  {\n    "name_origin": 2\n}            2
Batch 5 processed:
Batch 6 re

In [432]:
# batch_6745d2710bf881908e8e0549127c1cbb

,id,name_origin
0,2551077358424869,5
1,1790731198333911,3
2,8825664357498479,5
3,577447828281396,5
4,2071812146605669,5
...,...,...
2845,1012381103849512,1
2846,984268756796182,4
2847,1050639483461615,5
2848,854223829923469,5


In [505]:
all_results_df.to_csv(f"{output_folder}/all_results3.csv", index=False)


In [506]:
df_final = pd.merge(df, all_results_df[['id', 'name_origin']], on="id", how="left")


In [507]:
df_final

,id,username,name_origin
0,2708292252677311,Bryan Friedman talon_sportscards,NaN
1,8384736858321179,Pristine Auction pristineauction,NaN
2,1236200407598552,David John Pollard ohtanicanada,NaN
3,954864009785844,The 4th Quarter the4thquartershop,NaN
4,3858174901120088,The 4th Quarter the4thquartershop,NaN
...,...,...,...
7570,1333850967577362,TheRealDodgerHogs therealdodgerhogs,NaN
7571,1264224878366252,❤︎Sho0705Time❤︎ __sho0705time__,NaN
7572,563515409501409,Cinemonie thecinemonie,NaN
7573,911617997574545,David John Pollard ohtanicanada,NaN


In [508]:
df_final[df_final.name_origin.isna()==True]

,id,username,name_origin
0,2708292252677311,Bryan Friedman talon_sportscards,NaN
1,8384736858321179,Pristine Auction pristineauction,NaN
2,1236200407598552,David John Pollard ohtanicanada,NaN
3,954864009785844,The 4th Quarter the4thquartershop,NaN
4,3858174901120088,The 4th Quarter the4thquartershop,NaN
...,...,...,...
7570,1333850967577362,TheRealDodgerHogs therealdodgerhogs,NaN
7571,1264224878366252,❤︎Sho0705Time❤︎ __sho0705time__,NaN
7572,563515409501409,Cinemonie thecinemonie,NaN
7573,911617997574545,David John Pollard ohtanicanada,NaN


In [509]:
df_final.to_csv("final_results_usernamept3.csv", index=False)
